# 範例 17：Fine-Tuning 資料增強

使用 AI 來幫助生成更多訓練資料！

## 什麼是資料增強？

資料增強（Data Augmentation）是用少量的「種子資料」生成更多類似的訓練資料。

```
種子資料（5 筆）
      │
      ▼ AI 生成變體
增強後資料（50+ 筆）
```

## 為什麼需要資料增強？
- 人工準備大量資料很費時
- 增加訓練資料的多樣性
- 提高模型的泛化能力

## 學習目標
- 了解資料增強的概念
- 學會使用 AI 生成訓練資料
- 建立可擴展的資料集

## 前置需求
- LM Studio 運行中，Local Server 已啟動
- 安裝 openai 套件：`pip install openai`

## Step 1: 匯入套件並設定

In [ ]:
from openai import OpenAI
import json

client = OpenAI(
    base_url="http://localhost:1234/v1",
    api_key="not-needed"
)

print("已連接到 LM Studio！")

## Step 2: 準備種子資料

In [ ]:
# 種子資料：我們手動準備的高品質範例
seed_data = [
    {
        "question": "Python 的 list 和 tuple 有什麼差別？",
        "answer": "list 是可變的（mutable），可以新增、修改、刪除元素；tuple 是不可變的（immutable），建立後就不能改變。tuple 的效能比 list 好，適合用於不需要修改的資料。"
    },
    {
        "question": "什麼是迴圈？",
        "answer": "迴圈是讓程式重複執行某段程式碼的結構。Python 有兩種迴圈：for 迴圈用於遍歷序列，while 迴圈用於條件判斷。迴圈可以減少重複的程式碼，提高效率。"
    },
    {
        "question": "如何處理程式中的錯誤？",
        "answer": "Python 使用 try-except 來處理錯誤。把可能出錯的程式碼放在 try 區塊，錯誤處理放在 except 區塊。這樣程式遇到錯誤時不會直接停止，而是執行錯誤處理程式碼。"
    }
]

print(f"種子資料：{len(seed_data)} 筆")
for i, item in enumerate(seed_data, 1):
    print(f"\n{i}. Q: {item['question']}")

## Step 3: 建立資料增強函數

In [ ]:
def augment_qa_pair(question, answer, num_variations=3):
    """
    生成問答對的變體

    參數：
        question: 原始問題
        answer: 原始回答
        num_variations: 要生成的變體數量

    回傳：
        問答對變體列表
    """

    prompt = f"""請根據以下問答對，生成 {num_variations} 個類似但不同的問答變體。
保持回答的核心資訊相同，但改變問法和表達方式。

原始問題：{question}
原始回答：{answer}

請用以下 JSON 格式輸出：
[
    {{"question": "變體問題1", "answer": "變體回答1"}},
    {{"question": "變體問題2", "answer": "變體回答2"}},
    ...
]

注意：
1. 問題要用不同的方式表達同一個意思
2. 回答要包含相同的核心知識，但用詞不同
3. 使用繁體中文

只輸出 JSON，不要其他文字："""

    response = client.chat.completions.create(
        model="gpt-oss-120b",
        messages=[{"role": "user", "content": prompt}]
    )

    try:
        # 嘗試解析 JSON
        result_text = response.choices[0].message.content
        # 找出 JSON 部分（處理可能的額外文字）
        start = result_text.find('[')
        end = result_text.rfind(']') + 1
        if start != -1 and end != 0:
            json_text = result_text[start:end]
            variations = json.loads(json_text)
            return variations
        else:
            print("無法找到 JSON 格式")
            return []
    except json.JSONDecodeError as e:
        print(f"JSON 解析失敗：{e}")
        return []

## Step 4: 測試單一問答對的增強

In [ ]:
# 測試增強第一個問答對
test_q = seed_data[0]["question"]
test_a = seed_data[0]["answer"]

print(f"原始問題：{test_q}")
print(f"原始回答：{test_a[:50]}...")
print("\n正在生成變體...")

try:
    variations = augment_qa_pair(test_q, test_a, num_variations=2)
    
    print(f"\n生成了 {len(variations)} 個變體：")
    for i, var in enumerate(variations, 1):
        print(f"\n--- 變體 {i} ---")
        print(f"Q: {var.get('question', 'N/A')}")
        answer = var.get('answer', 'N/A')
        print(f"A: {answer[:80]}..." if len(answer) > 80 else f"A: {answer}")
except Exception as e:
    print(f"錯誤：{e}")

## Step 5: 增強整個資料集

In [ ]:
def augment_dataset(seed_data, variations_per_item=2):
    """
    增強整個資料集

    參數：
        seed_data: 種子資料（問答對列表）
        variations_per_item: 每筆資料生成的變體數

    回傳：
        增強後的資料集
    """
    augmented_data = []

    for i, item in enumerate(seed_data, 1):
        print(f"\n處理第 {i}/{len(seed_data)} 筆資料...")
        
        # 加入原始資料
        augmented_data.append(item)

        # 生成變體
        try:
            variations = augment_qa_pair(
                item["question"],
                item["answer"],
                variations_per_item
            )
            augmented_data.extend(variations)
            print(f"  生成了 {len(variations)} 個變體")
        except Exception as e:
            print(f"  生成失敗：{e}")

    return augmented_data

In [ ]:
# 增強資料集（這可能需要一些時間）
print("=== 開始資料增強 ===")
print(f"原始資料：{len(seed_data)} 筆")

try:
    augmented = augment_dataset(seed_data, variations_per_item=2)
    print(f"\n=== 增強完成 ===")
    print(f"增強後資料：{len(augmented)} 筆")
    print(f"增加了 {len(augmented) - len(seed_data)} 筆新資料！")
except Exception as e:
    print(f"增強過程出錯：{e}")
    augmented = seed_data  # 使用原始資料

## Step 6: 查看增強結果

In [ ]:
# 顯示所有資料
print("=== 增強後的完整資料集 ===")
for i, item in enumerate(augmented, 1):
    q = item.get('question', 'N/A')
    a = item.get('answer', 'N/A')
    print(f"\n[{i}]")
    print(f"Q: {q[:60]}..." if len(q) > 60 else f"Q: {q}")
    print(f"A: {a[:60]}..." if len(a) > 60 else f"A: {a}")

## Step 7: 儲存增強後的資料

In [ ]:
def save_augmented_dataset(data, filename):
    """
    儲存增強後的資料集為訓練格式
    """
    system_prompt = "你是一位程式設計教師，用簡單易懂的方式回答問題。"
    
    training_data = []
    for item in data:
        training_data.append({
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": item.get("question", "")},
                {"role": "assistant", "content": item.get("answer", "")}
            ]
        })
    
    with open(filename, "w", encoding="utf-8") as f:
        for item in training_data:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")
    
    print(f"已儲存 {len(training_data)} 筆訓練資料到 {filename}")

In [ ]:
# 儲存資料
save_augmented_dataset(augmented, "augmented_training_data.jsonl")

## 其他增強技巧

In [ ]:
def generate_similar_questions(topic, num_questions=5):
    """
    根據主題生成相關問題
    """
    prompt = f"""請生成 {num_questions} 個關於「{topic}」的常見問題。
這些問題應該是初學者可能會問的。

用以下 JSON 格式輸出：
["問題1", "問題2", ...]

只輸出 JSON："""
    
    response = client.chat.completions.create(
        model="gpt-oss-120b",
        messages=[{"role": "user", "content": prompt}]
    )
    
    try:
        result = response.choices[0].message.content
        start = result.find('[')
        end = result.rfind(']') + 1
        return json.loads(result[start:end])
    except:
        return []

In [ ]:
# 測試生成新問題
topic = "Python 函數"
print(f"生成關於「{topic}」的問題：\n")

try:
    new_questions = generate_similar_questions(topic, 5)
    for i, q in enumerate(new_questions, 1):
        print(f"{i}. {q}")
except Exception as e:
    print(f"生成失敗：{e}")

## 資料增強的注意事項

### 好的做法：
1. **保持核心資訊一致**：變體應該傳達相同的知識
2. **增加多樣性**：用不同方式表達同一概念
3. **檢查品質**：人工審核生成的資料

### 避免的錯誤：
1. **資訊失真**：變體改變了原始含義
2. **重複太多**：變體之間太相似
3. **品質下降**：盲目追求數量

## 重點回顧

1. **資料增強**：用 AI 生成訓練資料的變體

2. **優點**：
   - 節省人工準備資料的時間
   - 增加資料多樣性
   - 提高模型泛化能力

3. **流程**：
   - 準備種子資料 → AI 生成變體 → 檢查品質 → 儲存

4. **注意**：品質比數量更重要

## 下一步

在下一個範例中，我們將學習如何評估模型的效果！